In [1]:
import pandas as pd

In [2]:
# Create a dataset which only contains the FIPS, county name and state name

#county_df = pd.read_csv("./datatest/education_county_2023.csv", dtype={"FIPS Code": str})
#county_df = county_df[["FIPS Code", "County Name", "State Name"]]
#county_df.to_csv("datatest/data_county.csv", index = False)


In [3]:
# Get a complete list of all the counties in the US, to check the datasets from other sources 
county_df = pd.read_csv("./datatest/data_county.csv", dtype={"FIPS Code": str})

In [34]:
# Turn Fips Code into 5 digit string 
# df ... Dataframe to be modified
# fips_column ... Name of the column with FIPS Code
def turn_fips_into_string(df, fips_column):

    # Fill all rows with less than 5 characters with leading 0's
    df[fips_column] = df[fips_column].astype(str).str.zfill(5)

    if (df[fips_column].str.len() == 5).all():
        print("All FIPS Codes are now length 5.")
    else: 
        raise Exception("Not all FIPS Codes are length 5, check again.")

    return df  

# Modify the df 
# df ... Dataframe to be modified
# county_df ... Dataframe containing the FIPS Codes, State Name, County Name (imported from csv) 
# columns_to_keep ... dictionary with all the relevant variables 
columns_to_keep = {
    "variable_name": "RenamedVariable",
    #...
    }

def check_df(df, county_df, columns_to_keep):

    if "FIPS Code" not in columns_to_keep.values():
        raise Exception(f'Please provide a "FIPS Code" column in dictionary.')
    
    df = df[list(columns_to_keep.keys())].copy()

    df.rename(columns = columns_to_keep, inplace=True)
    
    # Merges only the rows where the df FIPS Code matches that of the county_df, otherwise NaN
    merged_df = pd.merge(county_df, df, on=["FIPS Code"], how ="left")

    missing_in_df = county_df[~county_df["FIPS Code"].isin(df["FIPS Code"])]

    if missing_in_df.shape[0] != 0:
        print(f"Some counties were missing in the dataframe, {missing_in_df.shape[0]} NaN values were introduced.")
        missing_fips = missing_in_df["FIPS Code"].tolist()
        print("Missing FIPS Codes:")
        print(missing_fips)

    missing_counts = merged_df.isnull().sum()
    print("Number of missing values in each column:")
    print(missing_counts)
    
    return merged_df  


In [76]:
# replace geo dataset with this: 
# https://public.opendatasoft.com/explore/dataset/georef-united-states-of-america-county/information/?disjunctive.ste_code&disjunctive.ste_name&disjunctive.coty_code&disjunctive.coty_name&sort=year


geo_df = pd.read_csv("./data/geo_county_2023.csv", sep=";", dtype={"Official Code County": str})

geo_df = geo_df.drop(["Geo Shape"], axis=1)

geo_df[["lat", "lng"]] = geo_df["Geo Point"].str.split(", " , expand=True)

geo_df["lat"] = geo_df["lat"].astype(float)
geo_df["lng"] = geo_df["lng"].astype(float)

geo_df = turn_fips_into_string(geo_df, "Official Code County")

geo_columns_to_keep = {
    "Official Code County": "FIPS Code",
    "lat": "Latitude",
    "lng": "Longitude"
}

geo_df = check_df(geo_df, county_df, geo_columns_to_keep)

geo_df.head()

All FIPS Codes are now length 5.
Number of missing values in each column:
FIPS Code      0
County Name    0
State Name     0
Latitude       0
Longitude      0
dtype: int64


,FIPS Code,County Name,State Name,Latitude,Longitude
0,01001,Autauga County,Alabama,32.534927,-86.642734
1,01003,Baldwin County,Alabama,30.727605,-87.722560
2,01005,Barbour County,Alabama,31.869595,-85.393235
3,01007,Bibb County,Alabama,32.998627,-87.126467
4,01009,Blount County,Alabama,33.980816,-86.567480


In [70]:
# https://www.ers.usda.gov/data-products/county-level-data-sets/county-level-data-sets-download-data

population_df = pd.read_csv("./data/population_county_2023.csv", encoding="latin1", dtype={"county_fips": str})

population_df = population_df[population_df["Attribute"] == "POP_ESTIMATE_2023"]

population_df.reset_index(drop=True, inplace=True)

population_df = turn_fips_into_string(population_df, "FIPStxt")

population_columns_to_keep = {
    "FIPStxt": "FIPS Code",
    "Value": "Population"
}

population_df = check_df(population_df, county_df, population_columns_to_keep)

population_df.head()

# R_BIRTH_2023
# R_DEATH_2023
# POP_ESTIMATE_2023

All FIPS Codes are now length 5.
Number of missing values in each column:
FIPS Code      0
County Name    0
State Name     0
Population     0
dtype: int64


,FIPS Code,County Name,State Name,Population
0,01001,Autauga County,Alabama,60342.0
1,01003,Baldwin County,Alabama,253507.0
2,01005,Barbour County,Alabama,24585.0
3,01007,Bibb County,Alabama,21868.0
4,01009,Blount County,Alabama,59816.0


In [59]:
poverty_df = pd.read_csv("./data/poverty_county_2023.csv", dtype={"county_fips": str})

poverty_df = poverty_df[poverty_df["Attribute"] == "PCTPOVALL_2023"]

poverty_df.reset_index(drop=True, inplace=True)

poverty_df = turn_fips_into_string(poverty_df, "FIPS_Code")

poverty_columns_to_keep = {
    "FIPS_Code": "FIPS Code",
    "Value": "Poverty Rate"
}

poverty_df = check_df(poverty_df, county_df, poverty_columns_to_keep)

poverty_df["Poverty Rate"] = poverty_df["Poverty Rate"] / 100

poverty_df.head()

All FIPS Codes are now length 5.
Some counties were missing in the dataframe, 1 NaN values were introduced.
Missing FIPS Codes:
['15005']
Number of missing values in each column:
FIPS Code       0
County Name     0
State Name      0
Poverty Rate    1
dtype: int64


,FIPS Code,County Name,State Name,Poverty Rate
0,01001,Autauga County,Alabama,0.117
1,01003,Baldwin County,Alabama,0.100
2,01005,Barbour County,Alabama,0.255
3,01007,Bibb County,Alabama,0.194
4,01009,Blount County,Alabama,0.128


In [60]:
unemployment_df = pd.read_csv("./data/unemployment_county_2023.csv", dtype={"county_fips": str})

unemployment_df = unemployment_df[unemployment_df["Attribute"] == "Unemployment_rate_2023"]

unemployment_df.reset_index(drop=True, inplace=True)

unemployment_df = turn_fips_into_string(unemployment_df, "FIPS_Code")

unemployment_columns_to_keep = {
    "FIPS_Code": "FIPS Code",
    "Value": "Unemployment Rate"
}

unemployment_df = check_df(unemployment_df, county_df, unemployment_columns_to_keep)

unemployment_df["Unemployment Rate"] = unemployment_df["Unemployment Rate"]  / 100

unemployment_df.head()

All FIPS Codes are now length 5.
Some counties were missing in the dataframe, 10 NaN values were introduced.
Missing FIPS Codes:
['09110', '09120', '09130', '09140', '09150', '09160', '09170', '09180', '09190', '15005']
Number of missing values in each column:
FIPS Code             0
County Name           0
State Name            0
Unemployment Rate    10
dtype: int64


,FIPS Code,County Name,State Name,Unemployment Rate
0,01001,Autauga County,Alabama,0.022
1,01003,Baldwin County,Alabama,0.023
2,01005,Barbour County,Alabama,0.044
3,01007,Bibb County,Alabama,0.025
4,01009,Blount County,Alabama,0.021


In [ ]:
ruralurban_df = pd.read_csv("./data/ruralurban_county_2023.csv", encoding="latin1", dtype={"FIPS": str})

ruralurban_df = ruralurban_df[ruralurban_df["Attribute"] == "RUCC_2023"]

ruralurban_df.reset_index(drop=True, inplace=True)

ruralurban_df = turn_fips_into_string(ruralurban_df, "FIPS")

ruralurban_columns_to_keep = {
    "FIPS": "FIPS Code",
    "Value": "RUCC"
}

ruralurban_df = check_df(ruralurban_df, county_df, ruralurban_columns_to_keep)

# RUCC Codes Classification 
# https://www.ers.usda.gov/data-products/rural-urban-continuum-codes/documentation
ruralurban_mapping = {1: "Metro", 2: "Metro", 3: "Metro",
           4: "Nonmetro", 5: "Nonmetro", 6: "Nonmetro",
           7: "Nonmetro", 8: "Nonmetro", 9: "Nonmetro"}

ruralurban_df["RUCC"] = ruralurban_df["RUCC"].astype(int)

ruralurban_df["AreaClassification"] = ruralurban_df["RUCC"].map(ruralurban_mapping)

ruralurban_df.head()

All FIPS Codes are now length 5.
Number of missing values in each column:
FIPS Code      0
County Name    0
State Name     0
RUCC           0
dtype: int64


,FIPS Code,County Name,State Name,RUCC,AreaClassification
0,01001,Autauga County,Alabama,2,Metro
1,01003,Baldwin County,Alabama,3,Metro
2,01005,Barbour County,Alabama,6,Nonmetro
3,01007,Bibb County,Alabama,1,Metro
4,01009,Blount County,Alabama,1,Metro


In [68]:
#2023-12-31

# StateCodeFIPS,MunicipalCodeFIPS

zillowhousevalue_df = pd.read_csv("./data/zillowhousevalue_county_2023.csv")

zillowhousevalue_df["FIPS Code"] = (
    zillowhousevalue_df["StateCodeFIPS"].astype(str).str.zfill(2) +
    zillowhousevalue_df["MunicipalCodeFIPS"].astype(str).str.zfill(3)
)

zillowhousevalue_months = ["2023-01-31","2023-02-28","2023-03-31","2023-04-30","2023-05-31","2023-06-30","2023-07-31","2023-08-31","2023-09-30","2023-10-31","2023-11-30","2023-12-31"]

zillowhousevalue_df["avg_zhvi_2023"] = zillowhousevalue_df[zillowhousevalue_months].mean(axis=1)

zillowhousevalue_df = turn_fips_into_string(zillowhousevalue_df, "FIPS Code")

zillowhousevalue_columns_to_keep = {
    "FIPS Code": "FIPS Code",
    "avg_zhvi_2023": "Avg 2023 ZHVI"
}

zillowhousevalue_df = check_df(zillowhousevalue_df, county_df, zillowhousevalue_columns_to_keep)

zillowhousevalue_df.head()

All FIPS Codes are now length 5.
Some counties were missing in the dataframe, 80 NaN values were introduced.
Missing FIPS Codes:
['02013', '02016', '02050', '02060', '02063', '02066', '02068', '02070', '02105', '02158', '02164', '02180', '02188', '02198', '02230', '02282', '02290', '05099', '08009', '08057', '09110', '09120', '09130', '09140', '09150', '09160', '09170', '09180', '09190', '15005', '16023', '28055', '28119', '29087', '29227', '31005', '31009', '31075', '32009', '35019', '35021', '35033', '38007', '38023', '38037', '38047', '38051', '38069', '38079', '38085', '38087', '38091', '46003', '46017', '46041', '46043', '46049', '46053', '46055', '46061', '46069', '46075', '46085', '46089', '46095', '46102', '46121', '46137', '48033', '48137', '48229', '48261', '48263', '48269', '48301', '48311', '48359', '48393', '48413', '49031']
Number of missing values in each column:
FIPS Code         0
County Name       0
State Name        0
Avg 2023 ZHVI    82
dtype: int64


,FIPS Code,County Name,State Name,Avg 2023 ZHVI
0,01001,Autauga County,Alabama,232121.156180
1,01003,Baldwin County,Alabama,374871.736532
2,01005,Barbour County,Alabama,149609.069373
3,01007,Bibb County,Alabama,202043.703575
4,01009,Blount County,Alabama,222756.257799


In [77]:
from functools import reduce

dfs = [geo_df, population_df, poverty_df, ruralurban_df, unemployment_df, zillowhousevalue_df]

merged_df = reduce(lambda left, right: pd.merge(left, right, on=["FIPS Code", "County Name", "State Name"], how="inner"), dfs)

print(merged_df.shape[0])

merged_df.head(200)

3144


,FIPS Code,County Name,State Name,Latitude,Longitude,Population,Poverty Rate,RUCC,AreaClassification,Unemployment Rate,Avg 2023 ZHVI
0,01001,Autauga County,Alabama,32.534927,-86.642734,60342.0,0.117,2,Metro,0.022,232121.156180
1,01003,Baldwin County,Alabama,30.727605,-87.722560,253507.0,0.100,3,Metro,0.023,374871.736532
2,01005,Barbour County,Alabama,31.869595,-85.393235,24585.0,0.255,6,Nonmetro,0.044,149609.069373
3,01007,Bibb County,Alabama,32.998627,-87.126467,21868.0,0.194,1,Metro,0.025,202043.703575
4,01009,Blount County,Alabama,33.980816,-86.567480,59816.0,0.128,1,Metro,0.021,222756.257799
...,...,...,...,...,...,...,...,...,...,...,...
195,06017,El Dorado County,California,38.778698,-120.524654,192215.0,0.070,1,Metro,0.040,634712.314109
196,06019,Fresno County,California,36.758230,-119.649266,1017162.0,0.177,1,Metro,0.075,376257.422283
197,06021,Glenn County,California,39.598143,-122.391979,28129.0,0.135,6,Nonmetro,0.063,337148.950899
198,06023,Humboldt County,California,40.699303,-123.875607,133985.0,0.167,5,Nonmetro,0.046,455115.885811


In [78]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3144 entries, 0 to 3143
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   FIPS Code           3144 non-null   object 
 1   County Name         3144 non-null   object 
 2   State Name          3144 non-null   object 
 3   Latitude            3144 non-null   float64
 4   Longitude           3144 non-null   float64
 5   Population          3144 non-null   float64
 6   Poverty Rate        3143 non-null   float64
 7   RUCC                3144 non-null   int64  
 8   AreaClassification  3144 non-null   object 
 9   Unemployment Rate   3134 non-null   float64
 10  Avg 2023 ZHVI       3062 non-null   float64
dtypes: float64(6), int64(1), object(4)
memory usage: 270.3+ KB


In [80]:
merged_df.to_csv("datanew/data_other_merged.csv", index = False)
